In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [2]:
## Creating a dataset for Pain related frames

class PainDataset(Dataset):
    
    def __init__(self):
        # Data Loading from csv files to numpy and torch tensors
        xy = np.loadtxt('C:/Users/Nischal/Desktop/BioVid_split/pain/Pain_features_and_labels_071309_w_21_pain.csv', delimiter=",",dtype=np.float32,skiprows=1)
        self.x=torch.from_numpy(xy[:7921,:-1])
        self.y=torch.from_numpy(xy[:,[-1]])
        self.n_samples = 7920 ## Adjusted total number of the frames to match as a multiple of features*sequence
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
    
    def __len__(self):
        # Len(dataset)
        return self.n_samples
        

In [3]:
# Test code to check length of dataset

# dataset = PainDataset()
# print(len(PainDataset()))
# dataloader = DataLoader(dataset=dataset, batch_size=10, shuffle= False)

# dataiter = iter(dataloader)
# data = dataiter.next()
# features, labels =data
# print(len(features[0]))

7920
18


In [7]:
## Main part of code, Building architecture and training for the Pain labels

# Device configuration for CPU or GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device = torch.device('cuda')

# Hyper-parameters 
# input_size = 18 AU (Both presence(1) and intensities(17) )
num_classes = 2 ## (Binary classification as pain(0) or no pain(1) )
num_epochs = 5
batch_size = 10
learning_rate = 0.001

input_size = 18
sequence_length = 10
hidden_size = 128
num_layers = 2


## PAIN DATASET

dataset = PainDataset()

train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)


# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # Can add sigmoid here itself ################################################## 
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # Check list sizes here to match required lengths
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 2)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer setting
# criterion = nn.CrossEntropyLoss() ## Useful for later stage where we might have to use pain levels
criterion = nn.BCELoss()
m = nn.Sigmoid()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


100
Epoch [1/5], Step [100/792], Loss: 0.0008
200
Epoch [1/5], Step [200/792], Loss: 0.0004
300
Epoch [1/5], Step [300/792], Loss: 0.0002
400
Epoch [1/5], Step [400/792], Loss: 0.0002
500
Epoch [1/5], Step [500/792], Loss: 0.0001
600
Epoch [1/5], Step [600/792], Loss: 0.0001
700
Epoch [1/5], Step [700/792], Loss: 0.0001
892
Epoch [2/5], Step [100/792], Loss: 0.0000
992
Epoch [2/5], Step [200/792], Loss: 0.0000
1092
Epoch [2/5], Step [300/792], Loss: 0.0000
1192
Epoch [2/5], Step [400/792], Loss: 0.0000
1292
Epoch [2/5], Step [500/792], Loss: 0.0000
1392
Epoch [2/5], Step [600/792], Loss: 0.0000
1492
Epoch [2/5], Step [700/792], Loss: 0.0000
1684
Epoch [3/5], Step [100/792], Loss: 0.0000
1784
Epoch [3/5], Step [200/792], Loss: 0.0000
1884
Epoch [3/5], Step [300/792], Loss: 0.0000
1984
Epoch [3/5], Step [400/792], Loss: 0.0000
2084
Epoch [3/5], Step [500/792], Loss: 0.0000
2184
Epoch [3/5], Step [600/792], Loss: 0.0000
2284
Epoch [3/5], Step [700/792], Loss: 0.0000
2476
Epoch [4/5], Step

In [ ]:
## Pain label training for the LSTM model

# Train the model
n_total_steps = len(train_loader)
output_list=[]
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        features = features.reshape(-1, sequence_length, input_size).to(device)
#         labels=torch.ones([2], dtype=torch.float32)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(features)
        output_list.append(outputs[0][0])
        loss = criterion(m(outputs), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(len(output_list))
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
# SAVING THE MODEL

# torch.save(model.state_dict(), 'path')
# # weights/xyz.pt
# model.load_state_dict(torch.load('path', map_location=torch.device('cpu')))

In [8]:
class NoPainDataset(Dataset):
    
    def __init__(self):
        # Data Loading
        xy = np.loadtxt('C:/Users/Nischal/Desktop/BioVid_split/no_pain/Pain_features_and_labels_071309_w_21_no_pain.csv', delimiter=",",dtype=np.float32,skiprows=1)
        self.x=torch.from_numpy(xy[:7921,:-1])
        self.y=torch.from_numpy(xy[:,[-1]])
        self.n_samples = 7920
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
    
    def __len__(self):
        # Len(dataset)
        return self.n_samples

In [9]:
dataset = NoPainDataset()

train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)

## No Pain label training for the LSTM model

# Train the model
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        features = features.reshape(-1, sequence_length, input_size).to(device)
#         labels=torch.ones([2], dtype=torch.float32)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(features)
#         print(outputs)
        loss = criterion(m(outputs), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/792], Loss: 0.0572
Epoch [1/5], Step [200/792], Loss: 0.0192
Epoch [1/5], Step [300/792], Loss: 0.0112
Epoch [1/5], Step [400/792], Loss: 0.0077
Epoch [1/5], Step [500/792], Loss: 0.0057
Epoch [1/5], Step [600/792], Loss: 0.0045
Epoch [1/5], Step [700/792], Loss: 0.0036
Epoch [2/5], Step [100/792], Loss: 0.0025
Epoch [2/5], Step [200/792], Loss: 0.0022
Epoch [2/5], Step [300/792], Loss: 0.0019
Epoch [2/5], Step [400/792], Loss: 0.0016
Epoch [2/5], Step [500/792], Loss: 0.0014
Epoch [2/5], Step [600/792], Loss: 0.0013
Epoch [2/5], Step [700/792], Loss: 0.0011
Epoch [3/5], Step [100/792], Loss: 0.0009
Epoch [3/5], Step [200/792], Loss: 0.0009
Epoch [3/5], Step [300/792], Loss: 0.0008
Epoch [3/5], Step [400/792], Loss: 0.0007
Epoch [3/5], Step [500/792], Loss: 0.0007
Epoch [3/5], Step [600/792], Loss: 0.0006
Epoch [3/5], Step [700/792], Loss: 0.0006
Epoch [4/5], Step [100/792], Loss: 0.0005
Epoch [4/5], Step [200/792], Loss: 0.0005
Epoch [4/5], Step [300/792], Loss:

In [11]:
## Testing the model for pain labels

dataset = PainDataset()
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for features, labels in test_loader:
        features = features.reshape(-1, sequence_length, input_size).to(device)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32).to(device)
#         labels=torch.ones([2], dtype=torch.float32)
        outputs = model(features)
        outputs=m(outputs)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on test data: {acc} %')

Accuracy of the network on test data: 100.0 %


In [12]:
## Testing the model for no pain labels

dataset = NoPainDataset()

train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for features, labels in test_loader:
        features = features.reshape(-1, sequence_length, input_size).to(device)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32).to(device)
        outputs = model(features)
        outputs=m(outputs)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on test data: {acc} %')

Accuracy of the network on test data: 0.0 %
